In [22]:
import numpy as np
import torch

# Example trajectories
trajectories = [
    [
        ([50, 950], [950, 50], (4, 2), 50, [150, 950], [950, 150]),
        ([150, 950], [950, 150], (4, 2), 30, [250, 950], [950, 250]),
        ([250, 950], [950, 250], (0, 3), 0, [250, 950], [850, 250]),
        ([250, 950], [850, 250], (1, 1), 20, [250, 850], [850, 150]),
        ([250, 850], [850, 150], (0, 4), 0, [250, 850], [950, 150]),
        ([250, 850], [950, 150], (1, 1), 20, [250, 750], [950, 50]),
        ([250, 750], [950, 50], (4, 1), -100, [350, 750], [950, 50]),
        ([350, 750], [950, 50], (1, 1), -80, [350, 650], [950, 50]),
        ([350, 650], [950, 50], (4, 0), 10, [450, 650], [950, 50]),
        ([450, 650], [950, 50], (4, 2), 30, [550, 650], [950, 150]),
        ([550, 650], [950, 150], (2, 4), -70, [550, 750], [950, 150]),
        ([550, 750], [950, 150], (0, 3), 0, [550, 750], [850, 150]),
        # # Add more steps...
    ],
    # Add more trajectories...
]


In [23]:
def preprocess_trajectories(trajectories):
    returns, states, actions, timesteps = [], [], [], []
    for traj in trajectories:
        traj_returns = []
        traj_states = []
        traj_actions = []
        traj_timesteps = []
        for t, step in enumerate(traj):
            state_uav1, state_uav2, action, reward, next_states, done = step
            state = state_uav1 + state_uav2
            traj_returns.append(reward)
            traj_states.append(state)
            traj_actions.append(list(action))
            traj_timesteps.append(t)  # Generate timesteps as a sequence of integers
        returns.append(traj_returns)
        states.append(traj_states)
        actions.append(traj_actions)
        timesteps.append(traj_timesteps)
    
    returns_array = np.array(returns)
    states_array = np.array(states)
    actions_array = np.array(actions)
    timesteps_array = np.array(timesteps)

    print(f'returns shape: {returns_array.shape}')
    print(f'states shape: {states_array.shape}')
    print(f'actions shape: {actions_array.shape}')
    print(f'timesteps shape: {timesteps_array.shape}')
    
    return returns_array, states_array, actions_array, timesteps_array

# Preprocess the data
returns, states, actions, timesteps = preprocess_trajectories(trajectories)

# Convert to PyTorch tensors
returns_tensor = torch.tensor(returns, dtype=torch.float32)
states_tensor = torch.tensor(states, dtype=torch.float32)
actions_tensor = torch.tensor(actions, dtype=torch.float32)
timesteps_tensor = torch.tensor(timesteps, dtype=torch.float32)

print(f'returns_tensor shape: {returns_tensor.shape}')  # Expected: (num_trajectories, num_steps)
print(f'states_tensor shape: {states_tensor.shape}')  # Expected: (num_trajectories, num_steps, state_dim)
print(f'actions_tensor shape: {actions_tensor.shape}')  # Expected: (num_trajectories, num_steps, action_dim)
print(f'timesteps_tensor shape: {timesteps_tensor.shape}')  # Expected: (num_trajectories, num_steps)


returns shape: (1, 12)
states shape: (1, 12, 4)
actions shape: (1, 12, 2)
timesteps shape: (1, 12)
returns_tensor shape: torch.Size([1, 12])
states_tensor shape: torch.Size([1, 12, 4])
actions_tensor shape: torch.Size([1, 12, 2])
timesteps_tensor shape: torch.Size([1, 12])


In [1]:
import numpy as np
import torch

# Example trajectories
# Each inner list is a sequence of (state, action, reward, next_state, done) tuples
trajectories = [
    [
        ([1.0, 2.0], [0.1, 0.2], 1.0, [1.5, 2.5], False),
        ([1.5, 2.5], [0.1, 0.2], 0.5, [2.0, 3.0], False),
        ([2.0, 3.0], [0.1, 0.2], -1.0, [2.5, 3.5], True)
    ],
    [
        ([0.5, 1.5], [0.2, 0.3], 1.5, [1.0, 2.0], False),
        ([1.0, 2.0], [0.2, 0.3], -0.5, [1.5, 2.5], True)
    ]
]

def preprocess_trajectories(trajectories, state_dim, action_dim, max_len):
    returns, states, actions, timesteps = [], [], [], []
    for traj in trajectories:
        traj_returns = []
        traj_states = []
        traj_actions = []
        traj_timesteps = []
        for t, step in enumerate(traj):
            state, action, reward, next_state, done = step
            traj_returns.append(reward)
            traj_states.append(state)
            traj_actions.append(action)
            traj_timesteps.append(t)
        
        # Pad sequences to the maximum length
        while len(traj_returns) < max_len:
            traj_returns.append(0.0)  # Assuming padding with 0 for rewards
            traj_states.append([0.0] * state_dim)  # Assuming padding with 0 for states
            traj_actions.append([0.0] * action_dim)  # Assuming padding with 0 for actions
            traj_timesteps.append(0)  # Assuming padding with 0 for timesteps
        
        returns.append(traj_returns)
        states.append(traj_states)
        actions.append(traj_actions)
        timesteps.append(traj_timesteps)
    
    returns_array = np.array(returns)
    states_array = np.array(states)
    actions_array = np.array(actions)
    timesteps_array = np.array(timesteps)
    
    # Calculate returns-to-go
    returns_to_go = []
    for traj in returns_array:
        traj_returns_to_go = np.flip(np.cumsum(np.flip(traj, axis=0)), axis=0)
        returns_to_go.append(traj_returns_to_go)
    
    returns_to_go_array = np.array(returns_to_go)
    
    print(f'returns shape: {returns_array.shape}')
    print(f'states shape: {states_array.shape}')
    print(f'actions shape: {actions_array.shape}')
    print(f'timesteps shape: {timesteps_array.shape}')
    print(f'returns_to_go shape: {returns_to_go_array.shape}')
    
    return returns_array, states_array, actions_array, timesteps_array, returns_to_go_array

# Define the dimensions and maximum length
state_dim = 2
action_dim = 2
max_len = 5  # Assuming a maximum length of 5 for simplicity

# Preprocess the data
returns, states, actions, timesteps, returns_to_go = preprocess_trajectories(trajectories, state_dim, action_dim, max_len)

# Convert to PyTorch tensors
returns_tensor = torch.tensor(returns, dtype=torch.float32)
states_tensor = torch.tensor(states, dtype=torch.float32)
actions_tensor = torch.tensor(actions, dtype=torch.float32)
timesteps_tensor = torch.tensor(timesteps, dtype=torch.float32)
returns_to_go_tensor = torch.tensor(returns_to_go, dtype=torch.float32)

print(f'returns_tensor shape: {returns_tensor.shape}')
print(f'states_tensor shape: {states_tensor.shape}')
print(f'actions_tensor shape: {actions_tensor.shape}')
print(f'timesteps_tensor shape: {timesteps_tensor.shape}')
print(f'returns_to_go_tensor shape: {returns_to_go_tensor.shape}')


returns shape: (2, 5)
states shape: (2, 5, 2)
actions shape: (2, 5, 2)
timesteps shape: (2, 5)
returns_to_go shape: (2, 5)
returns_tensor shape: torch.Size([2, 5])
states_tensor shape: torch.Size([2, 5, 2])
actions_tensor shape: torch.Size([2, 5, 2])
timesteps_tensor shape: torch.Size([2, 5])
returns_to_go_tensor shape: torch.Size([2, 5])


In [1]:
import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import activations

from decision_transformer.models.model import TrajectoryModel
from decision_transformer.models.trajectory_gpt2 import GPT2Model


class DecisionTransformer(TrajectoryModel):

    """
    This model uses GPT to model (Return_1, state_1, action_1, Return_2, state_2, ...)
    """

    def __init__(
            self,
            state_dim,
            act_dim,
            hidden_size,
            max_length=None,
            max_ep_len=4096,
            action_tanh=True,
            **kwargs
    ):
        super().__init__(state_dim, act_dim, max_length=max_length)

        self.hidden_size = hidden_size
        config = transformers.GPT2Config(
            vocab_size=1,  # doesn't matter -- we don't use the vocab
            n_embd=hidden_size,
            **kwargs
        )

        # note: the only difference between this GPT2Model and the default Huggingface version
        # is that the positional embeddings are removed (since we'll add those ourselves)
        self.transformer = GPT2Model(config)

        self.embed_timestep = nn.Embedding(max_ep_len, hidden_size)
        self.embed_return = torch.nn.Linear(1, hidden_size)
        self.embed_state = torch.nn.Linear(self.state_dim, hidden_size)
        self.embed_action = torch.nn.Linear(self.act_dim, hidden_size)

        self.embed_ln = nn.LayerNorm(hidden_size)

        # note: we don't predict states or returns for the paper
        self.predict_state = torch.nn.Linear(hidden_size, self.state_dim)
        self.predict_action = nn.Sequential(
            *([nn.Linear(hidden_size, self.act_dim)] + ([nn.Tanh()] if action_tanh else []))
        )
        self.predict_return = torch.nn.Linear(hidden_size, 1)

    def forward(self, states, actions, rewards, returns_to_go, timesteps, attention_mask=None):

        batch_size, seq_length = states.shape[0], states.shape[1]

        if attention_mask is None:
            # attention mask for GPT: 1 if can be attended to, 0 if not
            attention_mask = torch.ones((batch_size, seq_length), dtype=torch.long)

        # embed each modality with a different head
        state_embeddings = self.embed_state(states)
        action_embeddings = self.embed_action(actions)
        returns_embeddings = self.embed_return(returns_to_go)
        time_embeddings = self.embed_timestep(timesteps)

        # time embeddings are treated similar to positional embeddings
        state_embeddings = state_embeddings + time_embeddings
        action_embeddings = action_embeddings + time_embeddings
        returns_embeddings = returns_embeddings + time_embeddings

        # this makes the sequence look like (R_1, s_1, a_1, R_2, s_2, a_2, ...)
        # which works nice in an autoregressive sense since states predict actions
        stacked_inputs = torch.stack(
            (returns_embeddings, state_embeddings, action_embeddings), dim=1
        ).permute(0, 2, 1, 3).reshape(batch_size, 3*seq_length, self.hidden_size)
        stacked_inputs = self.embed_ln(stacked_inputs)

        # to make the attention mask fit the stacked inputs, have to stack it as well
        stacked_attention_mask = torch.stack(
            (attention_mask, attention_mask, attention_mask), dim=1
        ).permute(0, 2, 1).reshape(batch_size, 3*seq_length)

        # we feed in the input embeddings (not word indices as in NLP) to the model
        transformer_outputs = self.transformer(
            inputs_embeds=stacked_inputs,
            attention_mask=stacked_attention_mask,
        )
        x = transformer_outputs['last_hidden_state']

        # reshape x so that the second dimension corresponds to the original
        # returns (0), states (1), or actions (2); i.e. x[:,1,t] is the token for s_t
        x = x.reshape(batch_size, seq_length, 3, self.hidden_size).permute(0, 2, 1, 3)

        # get predictions
        return_preds = self.predict_return(x[:,2])  # predict next return given state and action
        state_preds = self.predict_state(x[:,2])    # predict next state given state and action
        action_preds = self.predict_action(x[:,1])  # predict next action given state

        return state_preds, action_preds, return_preds

    def get_action(self, states, actions, rewards, returns_to_go, timesteps, **kwargs):
        # we don't care about the past rewards in this model

        states = states.reshape(1, -1, self.state_dim)
        actions = actions.reshape(1, -1, self.act_dim)
        returns_to_go = returns_to_go.reshape(1, -1, 1)
        timesteps = timesteps.reshape(1, -1)

        if self.max_length is not None:
            states = states[:,-self.max_length:]
            actions = actions[:,-self.max_length:]
            returns_to_go = returns_to_go[:,-self.max_length:]
            timesteps = timesteps[:,-self.max_length:]

            # pad all tokens to sequence length
            attention_mask = torch.cat([torch.zeros(self.max_length-states.shape[1]), torch.ones(states.shape[1])])
            attention_mask = attention_mask.to(dtype=torch.long, device=states.device).reshape(1, -1)
            states = torch.cat(
                [torch.zeros((states.shape[0], self.max_length-states.shape[1], self.state_dim), device=states.device), states],
                dim=1).to(dtype=torch.float32)
            actions = torch.cat(
                [torch.zeros((actions.shape[0], self.max_length - actions.shape[1], self.act_dim),
                             device=actions.device), actions],
                dim=1).to(dtype=torch.float32)
            returns_to_go = torch.cat(
                [torch.zeros((returns_to_go.shape[0], self.max_length-returns_to_go.shape[1], 1), device=returns_to_go.device), returns_to_go],
                dim=1).to(dtype=torch.float32)
            timesteps = torch.cat(
                [torch.zeros((timesteps.shape[0], self.max_length-timesteps.shape[1]), device=timesteps.device), timesteps],
                dim=1
            ).to(dtype=torch.long)
        else:
            attention_mask = None

        _, action_preds, return_preds = self.forward(
            states, actions, None, returns_to_go, timesteps, attention_mask=attention_mask, **kwargs)

        return action_preds[0,-1]


c:\Users\steve\anaconda3\envs\ml_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: add_code_sample_docstrings() got an unexpected keyword argument 'tokenizer_class'

In [34]:
# Example input data dimensions
hidden_size = 128  # Hidden size for the transformer
max_length = 50  # Maximum sequence length
max_ep_len = 4096  # Maximum episode length

# Instantiate the model
model = DecisionTransformer(state_dim=state_dim, act_dim=action_dim, hidden_size=hidden_size, max_length=max_length, max_ep_len=max_ep_len)

# Forward pass through the model
state_preds, action_preds, return_preds = model(
    states=states_tensor,
    actions=actions_tensor,
    rewards=None,
    returns_to_go=returns_to_go_tensor,
    timesteps=timesteps_tensor
)

print(f'action_preds: {action_preds}')


AssertionError: 